In [10]:
import pandas as pd
import yfinance as yf

bist_data = pd.read_csv("/Users/muratisiklan/Desktop/stock-price/Artifacts/bist_data.csv")

In [11]:

def get_stock_data(symbol, start_date, end_date):
    try:
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

symbols = bist_data["Symbol"].head(5).to_list()


In [23]:
data = get_stock_data("BINHO.IS","2023-11-25","2023-12-30")

[*********************100%%**********************]  1 of 1 completed


In [24]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-21,235.699997,265.250000,224.000000,264.750000,264.750000,916841
2023-12-22,250.500000,278.000000,248.600006,259.250000,259.250000,648775
2023-12-26,230.000000,240.800003,222.600006,226.000000,226.000000,384567
2023-12-27,225.000000,235.699997,215.000000,225.000000,225.000000,400157
2023-12-28,223.000000,236.000000,221.000000,233.899994,233.899994,198613


In [21]:
from pymongo import MongoClient

# Constants
MONGO_URI = "mongodb://127.0.0.1:27017/"
DB_NAME = "stockdata"
COLLECTION_NAME = "test"

# Connect to MongoDB
with MongoClient(MONGO_URI) as client:
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]

    # Find the most recent entry
    most_recent_entry = collection.find_one({}, sort=[("_id", -1)])

# Print the _id field of the most recent entry
if most_recent_entry and "_id" in most_recent_entry:
    print("Most Recent Entry _id:", most_recent_entry["_id"])
else:
    print("No entries in the collection.")


Most Recent Entry _id: 2023-12-27


In [22]:
from pymongo import MongoClient

# Set up the MongoDB connection
mongo_uri = "mongodb://127.0.0.1:27017/"
db_name = "stockdata"
collection_name = "test"

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[db_name]
collection = db[collection_name]

# Find the most recent document based on the _id field
most_recent_document = collection.find_one(sort=[("_id", -1)])

# Print or process the most recent document
if most_recent_document:
    print("Most Recent Document:")
    print(most_recent_document)
else:
    print("No documents found in the collection.")


Most Recent Document:
{'_id': '2023-12-27', 'bist': [{'Symbol': 'BINHO.IS', 'Company': '1000 YATIRIMLAR HOLDİNG A.Ş.', 'Open': 225.0, 'High': 235.6999969482422, 'Low': 215.0, 'Close': 225.0, 'AdjClose': 225.0, 'Volume': 400157.0}, {'Symbol': 'AVOD.IS', 'Company': 'A.V.O.D. KURUTULMUŞ GIDA VE TARIM ÜRÜNLERİ SANAYİ TİCARET A.Ş.', 'Open': 2.9600000381469727, 'High': 3.069999933242798, 'Low': 2.950000047683716, 'Close': 3.0299999713897705, 'AdjClose': 3.0299999713897705, 'Volume': 6845574.0}, {'Symbol': 'A1CAP.IS', 'Company': 'A1 CAPITAL YATIRIM MENKUL DEĞERLER A.Ş.', 'Open': 24.760000228881836, 'High': 25.959999084472656, 'Low': 24.5, 'Close': 25.219999313354492, 'AdjClose': 25.219999313354492, 'Volume': 1281855.0}, {'Symbol': 'ACSEL.IS', 'Company': 'ACISELSAN ACIPAYAM SELÜLOZ SANAYİ VE TİCARET A.Ş.', 'Open': 93.0, 'High': 97.75, 'Low': 89.75, 'Close': 89.75, 'AdjClose': 89.75, 'Volume': 1330070.0}, {'Symbol': 'ADEL.IS', 'Company': 'ADEL KALEMCİLİK TİCARET VE SANAYİ A.Ş.', 'Open': 322.5, 

In [5]:
%pwd

'/Users/muratisiklan/Desktop/stock-price/Experiments'

In [6]:
import os
os.chdir("../")

In [7]:
from src.update_db import update_database

In [20]:
update_database("test")

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['BINHO.IS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-12-28 -> 2023-12-28)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['AVOD.IS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-12-28 -> 2023-12-28)')



[*********************100%%**********************]  1 of 1 completed

...


1 Failed download:
['XU030.IS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-12-28 -> 2023-12-28)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['GC=F']: IndexError('index 0 is out of bounds for axis 0 with size 0')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['XSIST.IS']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
